# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hamilton,39.1834,-84.5333,291.48,78,100,8.23,US,1709072163
1,1,kapuskasing,49.4169,-82.4331,271.20,93,100,4.12,CA,1709072231
2,2,jeddah,21.5169,39.2192,294.07,60,0,1.03,SA,1709072231
3,3,bredasdorp,-34.5322,20.0403,290.58,73,2,6.91,ZA,1709072232
4,4,teofilo otoni,-17.8575,-41.5053,296.38,88,51,1.15,BR,1709072233


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(x='Lng', y='Lat',  c='Humidity', 
                                      hover_cols=['City','Humidity'], tiles='OSM', colormap='viridis',
                                      geo=True,   size='Humidity', height=500, width=800, colorbar=True, legend= "right")



# Display the map
map_plot

ImportError: Geographic projection support requires: cartopy, geoviews, pyproj.

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_data_df = city_data_df[(city_data_df["Max Temp"] <= 300) & (city_data_df["Max Temp"] > 290) & 
                             (city_data_df["Wind Speed"] < 10) & city_data_df["Cloudiness"] <20]


# Drop any rows with null values
ideal_data_df.dropna()

# Display sample data
ideal_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hamilton,39.1834,-84.5333,291.48,78,100,8.23,US,1709072163
1,1,kapuskasing,49.4169,-82.4331,271.20,93,100,4.12,CA,1709072231
2,2,jeddah,21.5169,39.2192,294.07,60,0,1.03,SA,1709072231
3,3,bredasdorp,-34.5322,20.0403,290.58,73,2,6.91,ZA,1709072232
4,4,teofilo otoni,-17.8575,-41.5053,296.38,88,51,1.15,BR,1709072233
...,...,...,...,...,...,...,...,...,...,...
562,562,klaksvik,62.2266,-6.5890,276.66,72,42,15.41,FO,1709072653
563,563,shahr-e babak,30.1165,55.1186,278.80,62,25,2.09,IR,1709072654
564,564,cabo san lucas,22.8909,-109.9124,304.05,50,40,4.63,MX,1709072655
565,565,sao raimundo nonato,-9.0153,-42.6994,301.13,58,97,0.54,BR,1709072655


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
#Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= pd.DataFrame(city_data_df, columns=["City", "Country", "Lat", "Lng", "Humidity"])


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,hamilton,US,39.1834,-84.5333,78,
1,kapuskasing,CA,49.4169,-82.4331,93,
2,jeddah,SA,21.5169,39.2192,60,
3,bredasdorp,ZA,-34.5322,20.0403,73,
4,teofilo otoni,BR,-17.8575,-41.5053,88,
...,...,...,...,...,...,...
562,klaksvik,FO,62.2266,-6.5890,72,
563,shahr-e babak,IR,30.1165,55.1186,62,
564,cabo san lucas,MX,22.8909,-109.9124,50,
565,sao raimundo nonato,BR,-9.0153,-42.6994,58,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
target_coordinates = f"{hotel_df['Lat'][0]},{hotel_df['Lng'][0]}"
target_search = "hotel"
target_type = "hotel"
radius = 10000 

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
# Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    
    params = {"categories": "accommodation.hotel",   # This category includes hotels
              "filter": f"circle:{lng},{lat},{10000}",      # We further filter to only get hotels
              "limit": 1,   
              "apiKey": geoapify_key}

    
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
 
    # Make an API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
   
    
    # Convert the API response to JSON format
    name_address = name_address.json()
   
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except Exception as ex:# (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
# Display sample data
hotel_df

Starting hotel search
hamilton - nearest hotel: Homewood Suites
kapuskasing - nearest hotel: Travelodge
jeddah - nearest hotel: فندق درة الرؤية Pearl Of Vision Hotel
bredasdorp - nearest hotel: Victoria Hotel
teofilo otoni - nearest hotel: Hotel Beira Rio
waitangi - nearest hotel: Hotel Chathams
remire-montjoly - nearest hotel: Central Hôtel
aswan - nearest hotel: Mövenpick Resort Aswan
afaahiti - nearest hotel: Omati Lodge
bubaque - nearest hotel: Saldomar B&Biosphere
udachny - nearest hotel: Вилюй
edinburgh of the seven seas - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
kodiak - nearest hotel: Kodiak MWR Guest House
port-aux-francais - nearest hotel: Keravel
akjoujt - nearest hotel: فندق فاطمة
mount gambier - nearest hotel: Commodore on the Park
port alfred - nearest hotel: Ferryman
suva - nearest hotel: Wasawasa Hotel
bayanhongor - nearest hotel: No hotel found
yellowknife - nearest hotel: The Explorer Hotel
iwaki - nearest hotel: Hotel Hawaiians
tchintab

,City,Country,Lat,Lng,Humidity,Hotel Name
0,hamilton,US,39.1834,-84.5333,78,Homewood Suites
1,kapuskasing,CA,49.4169,-82.4331,93,Travelodge
2,jeddah,SA,21.5169,39.2192,60,فندق درة الرؤية Pearl Of Vision Hotel
3,bredasdorp,ZA,-34.5322,20.0403,73,Victoria Hotel
4,teofilo otoni,BR,-17.8575,-41.5053,88,Hotel Beira Rio
...,...,...,...,...,...,...
562,klaksvik,FO,62.2266,-6.5890,72,Hotel Klaksvik
563,shahr-e babak,IR,30.1165,55.1186,62,هتل شهرداری
564,cabo san lucas,MX,22.8909,-109.9124,50,Hotel Riu Santa Fe
565,sao raimundo nonato,BR,-9.0153,-42.6994,58,Pousada Progresso


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(x='Lng', y='Lat',  c='Humidity', 
                                    hover_cols=['City', 'Hotel Name',  'Humidity'], 
                                    tiles='OSM', colormap='viridis', geo=True,   
                                    size='Humidity', height=500, width=800, colorbar=True, legend= "right")


# Display the map
hotel_plot

ImportError: Geographic projection support requires: cartopy, geoviews, pyproj.